# UCLA model without visual attributes applied to the ITU + UCLA set

This notebook is aimed at running the UCLA original network without visual attributes to the combination of the ITU + UCLA datasets

In [2]:
from protestDB import cursor
import pandas as pd
from lib import analysis_utils as au

/home/lucas/Documents/ITU/Thesis_2018/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Loading up the data

In [3]:
pc = cursor.ProtestCursor()
imgs_UCLA = pc.getLabelledImagesAndNonProtest(source="UCLA", label_source="UCLA original")
imgs_ITU = pc.getLabelledImages(source="Luca Rossi - ECB")


print (len(imgs_UCLA), " UCLA images, both protest and non protest")
print (len(imgs_ITU), " ITU images, only protest related")


40720  UCLA images, both protest and non protest
1000  ITU images, only protest related
